In [ ]:
import pandas as pd
import numpy as np
from src.algorithm_functions import *

In [2]:
train_df = pd.read_csv('../Datasets/train_df.csv')
test_array = np.loadtxt('../Datasets/test_array.csv', delimiter=',')
train_array_weighted = fillna_means_weighted(train_df, 1/2, 1/2)
train_array = fillna_row_means(train_df)

In [8]:
train_array_zero = fillna_value(train_df, 0)
res = perform_svd2(train_df.isna(), train_array, test_array, r=16, max_iter=50)
res

(0.884367733227451, 50)